In [ ]:
# [CELL 1] Setup Environment
!pip install python-chess tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import chess
import chess.pgn
import os
import pickle
import shutil
import time
from tqdm import tqdm

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")
if device.type == 'cpu':
    print("⚠️ Warning: Bạn đang chạy trên CPU. Hãy bật GPU T4 trong Runtime > Change runtime type.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 62.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=9e8011aedc9cbd8a930cfa692f6e9480267280a673a4739fd3076e4ba3195c42
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess
✅ Device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# [CELL 2] Mount Drive & Setup Directories


# Định nghĩa đường dẫn làm việc trên Drive
# Bạn có thể đổi tên thư mục 'ChessAI' tùy ý
BASE_PATH = '/content/drive/MyDrive/Trí tuệ nhân tạo/BTL_2'
DATA_PATH = os.path.join(BASE_PATH, 'data_chess')
DATA_PATH = os.path.join(DATA_PATH, 'data_2000')
MODEL_PATH = os.path.join(BASE_PATH, 'weights')

# Tạo thư mục nếu chưa có
#os.makedirs(os.path.join(DATA_PATH, 'raw_pgn'), exist_ok=True)
os.makedirs(os.path.join(DATA_PATH, 'processed'), exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

print(f"📂 Working directory created at: {BASE_PATH}")

📂 Working directory created at: /content/drive/MyDrive/Trí tuệ nhân tạo/BTL_2


In [ ]:
# [CELL 3] Architecture: SmallResNet
class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        out = F.relu(out)
        return out

class SmallResNet(nn.Module):
    def __init__(self, num_res_blocks=6, num_channels=64, action_size=4672):
        super(SmallResNet, self).__init__()
        # Input: 32 channels (Current + History + Aux) [cite: 30]
        self.conv_input = nn.Conv2d(32, num_channels, kernel_size=3, padding=1, stride=1, bias=False)
        self.bn_input = nn.BatchNorm2d(num_channels)

        # Backbone: Residual Tower
        self.res_blocks = nn.ModuleList([
            ResidualBlock(num_channels) for _ in range(num_res_blocks)
        ])

        # Policy Head (Actor) [cite: 43]
        self.policy_conv = nn.Conv2d(num_channels, 32, kernel_size=1, stride=1, bias=False)
        self.policy_bn = nn.BatchNorm2d(32)
        self.policy_fc = nn.Linear(32 * 8 * 8, action_size)

        # Value Head (Critic) [cite: 44]
        self.value_conv = nn.Conv2d(num_channels, 3, kernel_size=1, stride=1, bias=False)
        self.value_bn = nn.BatchNorm2d(3)
        self.value_fc1 = nn.Linear(3 * 8 * 8, 64)
        self.value_fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.bn_input(self.conv_input(x)))
        for block in self.res_blocks:
            x = block(x)

        # Policy: Softmax
        p = F.relu(self.policy_bn(self.policy_conv(x)))
        p = p.view(-1, 32 * 8 * 8)
        p = self.policy_fc(p)
        policy_out = F.softmax(p, dim=1)

        # Value: Tanh [-1, 1]
        v = F.relu(self.value_bn(self.value_conv(x)))
        v = v.view(-1, 3 * 8 * 8)
        v = F.relu(self.value_fc1(v))
        v = torch.tanh(self.value_fc2(v))

        return policy_out, v

In [ ]:
# [CELL 4] Logic Core: ChessConverter
class ChessConverter:
    def __init__(self):
        # 12 channels for pieces (6 types * 2 colors)
        self.piece_map = {
            'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
            'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
        }
        self.move_to_idx = {}
        self.idx_to_move = {}
        self.next_idx = 0

    # Load/Save Map để đảm bảo nhất quán khi train nhiều lần
    def load_moves_map(self, path):
        if os.path.exists(path):
            with open(path, 'rb') as f:
                d = pickle.load(f)
                self.move_to_idx = d['move_to_idx']
                self.idx_to_move = d['idx_to_move']
                self.next_idx = len(self.move_to_idx)
            print(f"📖 Loaded Move Map: {self.next_idx} moves.")

    def save_moves_map(self, path):
        with open(path, 'wb') as f:
            pickle.dump({'move_to_idx': self.move_to_idx, 'idx_to_move': self.idx_to_move}, f)

    def encode_move(self, move_uci):
        # Tự động gán index cho nước đi mới
        if move_uci not in self.move_to_idx:
            if self.next_idx >= 4672: return None # Giới hạn output
            self.move_to_idx[move_uci] = self.next_idx
            self.idx_to_move[self.next_idx] = move_uci
            self.next_idx += 1
        return self.move_to_idx[move_uci]

    def board_to_tensor(self, board, prev_board=None):
        # Triển khai Tensor 32x8x8 theo tài liệu [cite: 30-38]
        tensor = np.zeros((32, 8, 8), dtype=np.float32)

        # Channel 0-11: Quân hiện tại
        for sq, pc in board.piece_map().items():
            tensor[self.piece_map[pc.symbol()]][chess.square_rank(sq)][chess.square_file(sq)] = 1

        # Channel 12-23: Quân quá khứ (History T-1)
        if prev_board:
            for sq, pc in prev_board.piece_map().items():
                tensor[self.piece_map[pc.symbol()]+12][chess.square_rank(sq)][chess.square_file(sq)] = 1

        # Channel 24-31: Phụ trợ (Turn, Castling, En-passant, Repetition)
        if board.turn == chess.WHITE: tensor[24,:,:] = 1
        if board.has_kingside_castling_rights(chess.WHITE): tensor[25,:,:] = 1
        if board.has_queenside_castling_rights(chess.WHITE): tensor[26,:,:] = 1
        if board.has_kingside_castling_rights(chess.BLACK): tensor[27,:,:] = 1
        if board.has_queenside_castling_rights(chess.BLACK): tensor[28,:,:] = 1
        if board.ep_square:
            tensor[29][chess.square_rank(board.ep_square)][chess.square_file(board.ep_square)] = 1
        if board.is_repetition(1): tensor[30,:,:] = 1
        if board.is_repetition(2): tensor[31,:,:] = 1

        return tensor

In [ ]:
# [CELL 5] Data Processing Pipeline
def process_data_pipeline(max_games=5000):
    #RAW_DIR = os.path.join(DATA_PATH, 'raw_pgn')
    OUTPUT_FILE = os.path.join(DATA_PATH, 'processed/combined_data_2000.npz')
    MAP_FILE = os.path.join(MODEL_PATH, 'move_map.pkl')

    # 1. Setup Converter
    converter = ChessConverter()
    converter.load_moves_map(MAP_FILE) # Load cũ nếu có

    states, policies, values = [], [], []
    pgn_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.pgn')]

    if not pgn_files:
        print("❌ Không tìm thấy file .pgn nào trong folder data/raw_pgn!")
        print("👉 Hãy upload file PGN lên Drive trước.")
        return

    print(f"🚀 Found {len(pgn_files)} files. Processing max {max_games} games...")

    total_games = 0
    for pgn_file in pgn_files:
        path = os.path.join(DATA_PATH, pgn_file)
        pgn = open(path)

        while total_games < max_games:
            try:
                game = chess.pgn.read_game(pgn)
            except: break
            if game is None: break

            # Chỉ lấy ván có kết quả rõ ràng
            res = game.headers.get("Result", "*")
            if res not in ["1-0", "0-1", "1/2-1/2"]: continue
            val = 1.0 if res == "1-0" else (-1.0 if res == "0-1" else 0.0)

            board = game.board()
            prev_board = None

            for move in game.mainline_moves():
                s = converter.board_to_tensor(board, prev_board)
                m = converter.encode_move(move.uci()) # Serialize move

                if m is not None:
                    states.append(s)
                    policies.append(m)
                    values.append(val)

                prev_board = board.copy()
                board.push(move)

            total_games += 1
            if total_games % 100 == 0: print(f"Processing... {total_games} games done.")

    # Save Data
    print("📦 Saving dataset...")
    np.savez_compressed(OUTPUT_FILE, states=np.array(states), policy_targets=np.array(policies), value_targets=np.array(values))
    converter.save_moves_map(MAP_FILE)
    print(f"✅ DONE! Saved to {OUTPUT_FILE}")

# Chạy xử lý (Chỉ cần chạy 1 lần khi có dữ liệu mới)
process_data_pipeline(max_games=8000)

📖 Loaded Move Map: 1908 moves.
🚀 Found 24 files. Processing max 8000 games...
Processing... 100 games done.
Processing... 200 games done.
Processing... 300 games done.
Processing... 400 games done.
Processing... 500 games done.
Processing... 600 games done.
Processing... 700 games done.
Processing... 800 games done.
Processing... 900 games done.
Processing... 1000 games done.
Processing... 1100 games done.
Processing... 1200 games done.
Processing... 1300 games done.
Processing... 1400 games done.
Processing... 1500 games done.
Processing... 1600 games done.
Processing... 1700 games done.
Processing... 1800 games done.
Processing... 1900 games done.
Processing... 2000 games done.
Processing... 2100 games done.
Processing... 2200 games done.
Processing... 2300 games done.
Processing... 2400 games done.
Processing... 2500 games done.
Processing... 2600 games done.
Processing... 2700 games done.
Processing... 2800 games done.
Processing... 2900 games done.
Processing... 3000 games done.
P

In [ ]:
# [CELL 6] High-Speed Dataset (RAM & Local SSD)
class InMemoryDataset(Dataset):
    def __init__(self, drive_path):
        # 1. Copy từ Drive -> Local (/content) để đọc siêu nhanh
        local_path = '/content/temp_data.npz'
        if os.path.exists(drive_path):
            print(f"⏳ Copying data from Drive to Local Disk... (Speed Boost)")
            shutil.copy(drive_path, local_path)
        else:
            print(f"❌ File not found: {drive_path}")
            # Tạo data giả để test nếu không có file thật
            self.generate_dummy()
            return

        # 2. Load toàn bộ vào RAM (Khắc phục lỗi zlib và chậm)
        print("⏳ Loading into RAM...")
        data = np.load(local_path)
        self.states = torch.from_numpy(data['states'])
        self.p_targets = torch.from_numpy(data['policy_targets']).long()
        self.v_targets = torch.from_numpy(data['value_targets']).float()
        print(f"✅ Loaded {len(self.states)} samples ready for training.")

    def generate_dummy(self):
        print("⚠️ Generating DUMMY data for testing.")
        self.states = torch.randn(1000, 32, 8, 8)
        self.p_targets = torch.randint(0, 4672, (1000,)).long()
        self.v_targets = torch.randint(-1, 2, (1000,)).float()

    def __len__(self):
        return len(self.states)

    def __getitem__(self, idx):
        return {
            'state': self.states[idx],
            'p_target': self.p_targets[idx],
            'v_target': self.v_targets[idx]
        }

In [ ]:
# [CELL 7] Training Loop
class AlphaZeroLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.nll = nn.NLLLoss() # CrossEntropy cho Policy

    def forward(self, p_pred, v_pred, p_target, v_target):
        # Loss = (z-v)^2 - pi^T*log(p) [cite: 16]
        v_loss = self.mse(v_pred.squeeze(), v_target)
        p_loss = self.nll(torch.log(p_pred + 1e-8), p_target)
        return v_loss + p_loss, v_loss, p_loss

def train(epochs=20, batch_size=256):
    DATA_FILE = os.path.join(DATA_PATH, 'processed/combined_data_2000.npz')
    MAP_FILE = os.path.join(MODEL_PATH, 'move_map.pkl')

    # Load Map để biết action_size
    converter = ChessConverter()
    converter.load_moves_map(MAP_FILE)
    action_size = max(converter.next_idx, 4672)

    # Setup Dataset (Fast Mode)
    dataset = InMemoryDataset(DATA_FILE)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0) # num_workers=0 vì data đã ở RAM

    # Model Setup
    model = SmallResNet(action_size=action_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = AlphaZeroLoss()

    best_loss = float('inf')

    print("🚀 START TRAINING...")
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        loop = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}")

        for batch in loop:
            states = batch['state'].to(device)
            p_t = batch['p_target'].to(device)
            v_t = batch['v_target'].to(device)

            optimizer.zero_grad()
            p_pred, v_pred = model(states)

            loss, v_l, p_l = criterion(p_pred, v_pred, p_t, v_t)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(loader)
        print(f"Epoch {epoch+1} finished. Avg Loss: {avg_loss:.4f}")

        # Save Checkpoint
        torch.save(model.state_dict(), os.path.join(MODEL_PATH, 'model_latest_2000.pth'))
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), os.path.join(MODEL_PATH, 'model_best_2000.pth'))
            print("🌟 Saved new BEST model.")

# RUN TRAINING
train(epochs=20)

In [ ]:
# [CELL FINE-TUNE 1] Process NEW 8000 Games
# Đặt tên file đầu ra khác đi để phân biệt với đợt 1
NEW_DATA_FILE = os.path.join(DATA_PATH, 'processed/batch_4_data.npz')

def process_new_batch(max_games=8000):
    RAW_DIR = os.path.join(DATA_PATH, 'dt')
    MAP_FILE = os.path.join(MODEL_PATH, 'move_map.pkl') # BẮT BUỘC DÙNG MAP CŨ

    # 1. Load Converter với Map cũ
    converter = ChessConverter()
    if os.path.exists(MAP_FILE):
        converter.load_moves_map(MAP_FILE)
        print(f"📖 Loaded existing map with {converter.next_idx} moves. Ready to extend.")
    else:
        print("⚠️ CRITICAL WARNING: Không tìm thấy move_map.pkl cũ! Model sẽ bị lỗi index.")
        return

    states, policies, values = [], [], []
    pgn_files = [f for f in os.listdir(RAW_DIR) if f.endswith('.pgn')]

    print(f"🚀 Processing new batch from: {pgn_files}")

    total_games = 0
    for pgn_file in pgn_files:
        path = os.path.join(RAW_DIR, pgn_file)
        pgn = open(path)

        while total_games < max_games:
            try:
                game = chess.pgn.read_game(pgn)
            except: break
            if game is None: break

            res = game.headers.get("Result", "*")
            if res not in ["1-0", "0-1", "1/2-1/2"]: continue
            val = 1.0 if res == "1-0" else (-1.0 if res == "0-1" else 0.0)

            board = game.board()
            prev_board = None

            for move in game.mainline_moves():
                s = converter.board_to_tensor(board, prev_board)
                m = converter.encode_move(move.uci())

                if m is not None:
                    states.append(s)
                    policies.append(m)
                    values.append(val)
                prev_board = board.copy()
                board.push(move)

            total_games += 1
            if total_games % 500 == 0: print(f"  -> Processed {total_games} games...")

    # Lưu dữ liệu Batch 2
    print(f"📦 Saving Batch 2 dataset ({len(states)} samples)...")
    np.savez_compressed(NEW_DATA_FILE, states=np.array(states), policy_targets=np.array(policies), value_targets=np.array(values))

    # Cập nhật lại Map (nếu có nước đi mới lạ xuất hiện trong 8000 ván này)
    converter.save_moves_map(MAP_FILE)
    print(f"✅ DONE! Saved to {NEW_DATA_FILE}")

# Chạy xử lý batch mới
process_new_batch(max_games=8000)

📖 Loaded Move Map: 1905 moves.
📖 Loaded existing map with 1905 moves. Ready to extend.
🚀 Processing new batch from: ['ficsgamesdb_2012_standard_nomovetimes_925737.pgn', 'ficsgamesdb_2011_standard_nomovetimes_925886.pgn']
  -> Processed 500 games...
  -> Processed 1000 games...
  -> Processed 1500 games...
  -> Processed 2000 games...
  -> Processed 2500 games...
  -> Processed 3000 games...
  -> Processed 3500 games...
  -> Processed 4000 games...
  -> Processed 4500 games...
  -> Processed 5000 games...
  -> Processed 5500 games...
  -> Processed 6000 games...
  -> Processed 6500 games...
  -> Processed 7000 games...
  -> Processed 7500 games...
  -> Processed 8000 games...
📦 Saving Batch 2 dataset (502346 samples)...
✅ DONE! Saved to /content/drive/MyDrive/Trí tuệ nhân tạo/BTL_2/data_chess/processed/batch_4_data.npz


In [ ]:
# [CELL FINE-TUNE 2] Load Weights & Train
def fine_tune_model(epochs=10, batch_size=256):
    # Đường dẫn file Batch 2 mới tạo
    NEW_DATA_FILE = os.path.join(DATA_PATH, 'processed/batch_4_data.npz')
    MAP_FILE = os.path.join(MODEL_PATH, 'move_map.pkl')
    PREV_MODEL_PATH = os.path.join(MODEL_PATH, 'model_best.pth') # Model đợt 1

    # 1. Setup Resources
    converter = ChessConverter()
    converter.load_moves_map(MAP_FILE)
    action_size = max(converter.next_idx, 4672) # Đảm bảo size khớp với map đã update

    # 2. Load New Dataset
    dataset = InMemoryDataset(NEW_DATA_FILE) # Class này đã định nghĩa ở các cell trước
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    # 3. Initialize Model
    model = SmallResNet(action_size=action_size).to(device)

    # --- PHẦN QUAN TRỌNG NHẤT: LOAD WEIGHTS ---
    if os.path.exists(PREV_MODEL_PATH):
        print(f"🔄 Loading weights from previous training: {PREV_MODEL_PATH}")
        # load_state_dict: nạp toàn bộ tham số w, b cũ vào mạng mới
        # strict=False: cho phép bỏ qua lỗi nhỏ nếu output size bị thay đổi nhẹ (do thêm nước đi mới)
        try:
            model.load_state_dict(torch.load(PREV_MODEL_PATH, map_location=device), strict=False)
            print("✅ Weights loaded successfully! Fine-tuning started.")
        except Exception as e:
            print(f"⚠️ Warning loading weights: {e}")
            print("Tip: Nếu lỗi dimension, có thể do move_map bị reset. Hãy kiểm tra lại.")
    else:
        print("⚠️ Không tìm thấy model cũ. Sẽ train từ đầu (Scratch).")

    # 4. Optimizer với Learning Rate THẤP HƠN
    # Giảm LR xuống 1e-4 (so với 1e-3 lúc đầu) để tinh chỉnh nhẹ nhàng
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    criterion = AlphaZeroLoss()

    best_loss = float('inf')

    # 5. Training Loop (Giống hệt cũ)
    print("🚀 START FINE-TUNING...")
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        loop = tqdm(loader, desc=f"Fine-tune Epoch {epoch+1}/{epochs}")

        for batch in loop:
            states = batch['state'].to(device)
            p_t = batch['p_target'].to(device)
            v_t = batch['v_target'].to(device)

            optimizer.zero_grad()
            p_pred, v_pred = model(states)

            loss, v_l, p_l = criterion(p_pred, v_pred, p_t, v_t)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(loader)
        print(f"Epoch {epoch+1} finished. Avg Loss: {avg_loss:.4f}")

        # Lưu đè lên model best hiện tại nếu tốt hơn
        torch.save(model.state_dict(), os.path.join(MODEL_PATH, 'model_latest.pth'))
        if avg_loss < best_loss:
            best_loss = avg_loss
            # Bạn có thể lưu thành tên khác như 'model_finetuned.pth' nếu muốn giữ file cũ
            torch.save(model.state_dict(), os.path.join(MODEL_PATH, 'model_best.pth'))
            print("🌟 Checkpoint updated.")

# Chạy Fine-tune
fine_tune_model(epochs=10) # 10 Epochs là đủ cho fine-tuning

📖 Loaded Move Map: 1908 moves.
⏳ Copying data from Drive to Local Disk... (Speed Boost)
⏳ Loading into RAM...
✅ Loaded 502346 samples ready for training.
🔄 Loading weights from previous training: /content/drive/MyDrive/Trí tuệ nhân tạo/BTL_2/weights/model_best.pth
✅ Weights loaded successfully! Fine-tuning started.
🚀 START FINE-TUNING...


Fine-tune Epoch 1/10: 100%|██████████| 1963/1963 [00:53<00:00, 36.72it/s, loss=3.36]


Epoch 1 finished. Avg Loss: 3.8943
🌟 Checkpoint updated.


Fine-tune Epoch 2/10: 100%|██████████| 1963/1963 [00:52<00:00, 37.39it/s, loss=2.86]


Epoch 2 finished. Avg Loss: 3.1142
🌟 Checkpoint updated.


...

Epoch 8 finished. Avg Loss: 1.9058
🌟 Checkpoint updated.


Fine-tune Epoch 9/10: 100%|██████████| 1963/1963 [00:52<00:00, 37.74it/s, loss=1.77]


Epoch 9 finished. Avg Loss: 1.7794
🌟 Checkpoint updated.


Fine-tune Epoch 10/10: 100%|██████████| 1963/1963 [00:52<00:00, 37.66it/s, loss=1.82]


Epoch 10 finished. Avg Loss: 1.6660
🌟 Checkpoint updated.


In [ ]:
import chess
import chess.svg
import torch
import numpy as np
import random
import time
import os
from IPython.display import display, clear_output

# --- 1. ĐỊNH NGHĨA AGENT ---

class RandomAgent:
    def select_move(self, board):
        # Chọn ngẫu nhiên 1 nước trong các nước hợp lệ
        moves = list(board.legal_moves)
        return random.choice(moves) if moves else None

class NeuralAgent:
    def __init__(self, model_path, map_path, device):
        self.device = device

        # Load Converter & Map
        self.converter = ChessConverter()
        self.converter.load_moves_map(map_path)

        # Load Model
        action_size = max(self.converter.next_idx, 4672)
        self.model = SmallResNet(action_size=action_size).to(device)
        # Load weights (chấp nhận strict=False để tránh lỗi nhỏ về size)
        self.model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
        self.model.eval()

    def select_move(self, board):
        # 1. Chuyển đổi bàn cờ sang Tensor
        state = self.converter.board_to_tensor(board)
        state_t = torch.tensor(state).unsqueeze(0).to(self.device) # [1, 32, 8, 8]

        # 2. Predict
        with torch.no_grad():
            policy, value = self.model(state_t)

        # 3. Lọc nước đi hợp lệ (Masking)
        legal_moves = list(board.legal_moves)
        best_move = None
        best_prob = -1.0

        # Duyệt qua các nước hợp lệ để tìm nước có xác suất cao nhất theo Neural Net
        policy_np = policy.cpu().numpy()[0] # Vector xác suất

        for move in legal_moves:
            move_uci = move.uci()
            # Lấy index của nước đi từ map
            idx = self.converter.move_to_idx.get(move_uci, None)

            if idx is not None and idx < len(policy_np):
                prob = policy_np[idx]
                if prob > best_prob:
                    best_prob = prob
                    best_move = move

        # Fallback: Nếu không tìm thấy nước nào trong map (hiếm), chọn random
        if best_move is None and legal_moves:
            print("⚠️ Model confused (Move not in map). Random pick.")
            return random.choice(legal_moves)

        return best_move

# --- 2. HÀM CHẠY ĐẤU (ARENA) ---

def play_match(model_path, map_path, num_games=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🏟️ Starting Arena on {device}...")

    # Khởi tạo đấu thủ
    neural_player = NeuralAgent(model_path, map_path, device)
    random_player = RandomAgent()

    results = {"Neural_Win": 0, "Random_Win": 0, "Draw": 0}

    for i in range(num_games):
        board = chess.Board()

        # Đổi màu quân sau mỗi ván để công bằng
        # Ván chẵn: Neural cầm Trắng (White)
        # Ván lẻ: Neural cầm Đen (Black)
        neural_is_white = (i % 2 == 0)

        print(f"\n--- Game {i+1}/{num_games} ---")
        print(f"Neural plays {'White' if neural_is_white else 'Black'}")

        move_count = 0
        while not board.is_game_over(claim_draw=True):
            # Xác định lượt đi
            if board.turn == chess.WHITE:
                current_player = neural_player if neural_is_white else random_player
            else:
                current_player = random_player if neural_is_white else neural_player

            # Agent chọn nước đi
            move = current_player.select_move(board)
            board.push(move)
            move_count += 1

            # (Tùy chọn) In bàn cờ mỗi 10 nước để đỡ lag log
            # if move_count % 10 == 0: print(".", end="")

        # Kết thúc ván
        res = board.result()
        print(f"\nGame Over: {res} (Moves: {move_count})")

        # Phân định thắng thua
        if res == "1-0":
            winner = "Neural" if neural_is_white else "Random"
        elif res == "0-1":
            winner = "Random" if neural_is_white else "Neural"
        else:
            winner = "Draw"

        if winner == "Neural": results["Neural_Win"] += 1
        elif winner == "Random": results["Random_Win"] += 1
        else: results["Draw"] += 1

        print(f"Winner: {winner}")

    print("\n================ RESULT ================")
    print(f"Neural Win: {results['Neural_Win']}")
    print(f"Random Win: {results['Random_Win']}")
    print(f"Draw:       {results['Draw']}")
    print("========================================")

# --- 3. THỰC THI ---
# Đảm bảo đường dẫn đúng
MODEL_FILE = os.path.join(MODEL_PATH, 'model_best_2000.pth')
MAP_FILE = os.path.join(MODEL_PATH, 'move_map.pkl')

if os.path.exists(MODEL_FILE):
    play_match(MODEL_FILE, MAP_FILE, num_games=10)
else:
    print("❌ Chưa có file model. Hãy train trước!")

In [ ]:
import chess
import torch
import numpy as np
import os
import random
from tqdm import tqdm  # Thanh tiến trình
import time

# --- CẤU HÌNH ---
NUM_GAMES = 1000          # Số lượng ván
MAX_MOVES_PER_GAME = 150  # Giới hạn nước đi để tránh treo máy (Xử hòa nếu vượt quá)
MODEL_FILE = os.path.join(MODEL_PATH, 'model_best_2000.pth')
MAP_FILE = os.path.join(MODEL_PATH, 'move_map.pkl')

# --- ĐỊNH NGHĨA LẠI AGENT (Để đảm bảo tính nhất quán) ---
class RandomAgent:
    def select_move(self, board):
        moves = list(board.legal_moves)
        return random.choice(moves) if moves else None

class NeuralAgent:
    def __init__(self, model_path, map_path, device):
        self.device = device
        self.converter = ChessConverter()
        self.converter.load_moves_map(map_path)

        # Load Model
        action_size = max(self.converter.next_idx, 4672)
        self.model = SmallResNet(action_size=action_size).to(device)

        if os.path.exists(model_path):
            self.model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
            self.model.eval()
        else:
            raise FileNotFoundError("❌ Không tìm thấy file model!")

    def select_move(self, board):
        # Biến đổi bàn cờ
        state = self.converter.board_to_tensor(board)
        state_t = torch.tensor(state).unsqueeze(0).to(self.device)

        with torch.no_grad():
            policy, _ = self.model(state_t)

        legal_moves = list(board.legal_moves)
        policy_np = policy.cpu().numpy()[0]

        best_move = None
        best_prob = -1.0

        # Lấy nước đi tốt nhất trong các nước hợp lệ
        for move in legal_moves:
            idx = self.converter.move_to_idx.get(move.uci(), None)
            if idx is not None and idx < len(policy_np):
                if policy_np[idx] > best_prob:
                    best_prob = policy_np[idx]
                    best_move = move

        # Fallback random nếu gặp lỗi lạ
        return best_move if best_move else random.choice(legal_moves)

# --- HÀM CHẠY BENCHMARK ---
def run_benchmark():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Starting Benchmark: {NUM_GAMES} games on {device}...")

    try:
        neural_bot = NeuralAgent(MODEL_FILE, MAP_FILE, device)
        random_bot = RandomAgent()
    except Exception as e:
        print(f"❌ Error loading agent: {e}")
        return

    stats = {"Win": 0, "Loss": 0, "Draw": 0}
    start_time = time.time()

    # Dùng tqdm để hiện thanh loading
    for i in tqdm(range(NUM_GAMES), desc="Simulating"):
        board = chess.Board()

        # Chẵn: Neural cầm Trắng | Lẻ: Neural cầm Đen
        neural_is_white = (i % 2 == 0)

        white_player = neural_bot if neural_is_white else random_bot
        black_player = random_bot if neural_is_white else neural_bot

        move_count = 0
        game_result = None # None: chưa xong

        # Vòng lặp ván đấu
        while not board.is_game_over():
            if move_count >= MAX_MOVES_PER_GAME:
                game_result = "Draw (Timeout)" # Xử hòa nếu đánh quá lâu
                break

            if board.turn == chess.WHITE:
                move = white_player.select_move(board)
            else:
                move = black_player.select_move(board)

            board.push(move)
            move_count += 1

        # Xác định kết quả
        if game_result is None:
            res = board.result() # "1-0", "0-1", "1/2-1/2"
        else:
            res = "1/2-1/2" # Timeout coi như hòa

        # Tính điểm cho Neural Network
        if res == "1-0":
            if neural_is_white: stats["Win"] += 1
            else: stats["Loss"] += 1
        elif res == "0-1":
            if neural_is_white: stats["Loss"] += 1
            else: stats["Win"] += 1
        else:
            stats["Draw"] += 1

    total_time = time.time() - start_time

    # --- HIỂN THỊ BÁO CÁO ---
    print("\n" + "="*40)
    print(f"📊 BENCHMARK REPORT ({NUM_GAMES} Games)")
    print("="*40)
    print(f"🏆 WINS:   {stats['Win']} ({stats['Win']/NUM_GAMES*100:.2f}%)")
    print(f"❌ LOSSES: {stats['Loss']} ({stats['Loss']/NUM_GAMES*100:.2f}%)")
    print(f"🤝 DRAWS:  {stats['Draw']} ({stats['Draw']/NUM_GAMES*100:.2f}%)")
    print("-" * 40)
    print(f"⏱️ Total Time: {total_time:.1f}s")
    print(f"⚡ Speed: {NUM_GAMES/total_time:.2f} games/sec")
    print("="*40)

# Chạy luôn
run_benchmark()

🚀 Starting Benchmark: 1000 games on cuda...
📖 Loaded Move Map: 1914 moves.


Simulating: 100%|██████████| 1000/1000 [01:40<00:00,  9.92it/s]


📊 BENCHMARK REPORT (1000 Games)
🏆 WINS:   576 (57.60%)
❌ LOSSES: 1 (0.10%)
🤝 DRAWS:  423 (42.30%)
----------------------------------------
⏱️ Total Time: 100.8s
⚡ Speed: 9.92 games/sec
